# 3.1 - Macrobond web API - Aligning multiple Time Series

*Using Macrobond's web API features to align various time series on a single calendar, frequency or currency and deal with missing values when observations do not all carry the same frequency.*

This notebook aims to provide examples of how to use Macrobond's web API call methods as well as insights on the various methodologies used to align our time series for analysis.

We will focus here on using the FetchUnifiedSeries POST call. This helps you doing the necessary pre-work before running your analysis or model.

*The examples uses the Web API, but you can chose to use the desktop COM API and get the same result. Full error handling is omitted for brevity*

***

## Importing packages

In [4]:
import statsmodels.api as statsmodels_api
from sklearn import linear_model

from macrobond_financial.web import WebClient
from macrobond_financial.common.types import StartOrEndPoint
from macrobond_financial.common.enums import SeriesFrequency

***

## Get the data - fetchunifiedseries

Note that we are using here the below time series in this example:
* cyinea0001 - Cyprus, Earnings, Wage Growth, Nominal
* cypric0014 - Cyprus, Consumer Price Index, Miscellaneous Goods & Services, Index
* cytour0076 - Cyprus, Income, Revenue, Total, EUR
* un_myos_cy_total - Cyprus, Human Development, Education, Mean Years of Schooling

Feel free to refer to https://api.macrobondfinancial.com/swagger/index.html to get the comprehensive list of web API endpoints and parameters used.

We want to look at data from Cyprus and conduct multiple regression analysis further down the notebook. Our dataset has the following features:

•	Our dependant variable will be nominal wage growth, which has an inception date of 1960 and is collected from the Cyprus Statistical Service (CYSTAT), the frequency is annual.

•	Our first independent variable will be Consumer Price Index for Miscellaneous Goods & Services which has an inception date of 2000 and is also collected from CYSTAT, and the frequency is monthly.

•	Our second independent variable is Income, total Revenue from foreign tourism (EUR) which has an inception date of 2001 and collected from CYSTAT, it has a monthly frequency.

•	Our final independent variable is Education, Mean Years of Schooling which is collected from United Nations Development Programme (UNDP) and has an inception date of 1990, the frequency is annual.
Immediately we can see that the current data set have different time scales, frequencies and currencies. So, in order for us to make the data comparable, we will utilise the 'Fetchallunifiedseries' endpoint which utilises a post request, which takes the following form below, let's see what each of these mean and how it can manipulate our data.

***

## Visualising the data
We have flattened the columns we want to portray in our chart (dates and values) to prepare our data to be graphed further down the notebook

In [ ]:
with WebClient() as api:
    data_frame = api.get_unified_series(
        "cyinea0001",
        "cypric0014",
        "cytour0076",
        "un_myos_cy_total",
        frequency=SeriesFrequency.ANNUAL,
        currency="USD",
        start_point=StartOrEndPoint.data_in_all_series(),
        end_point=StartOrEndPoint.data_in_all_series(),
    ).to_pd_data_frame()
data_frame.columns = [
    "Wage Growth",
    "CPI",
    "Income from Foreign Tourism",
    "Mean Years of Schooling"
]
data_frame

***

## Multiple Regression Analysis

Now that we have all the variables visually, we will use the package sklearn and from there use the linear_model package to make our model. Let us first start by defining our variables.

In [ ]:
x = data_frame[["CPI", "Income from Foreign Tourism", "Mean Years of Schooling"]]
y = data_frame["Wage Growth"]

regr = linear_model.LinearRegression()
regr.fit(x, y)

x = statsmodels_api.add_constant(x)
Summary = statsmodels_api.OLS(y, x).fit()
Summary.summary()

In [ ]:
CYP_Wage_Growth = regr.predict([[100.010000, 2.994805e09, 12.1712]])
print("Cyprus Wage Growth Forecast")
print(CYP_Wage_Growth)

***

## Conclusion

Here we can see how the FetchUnifiedSeries endpoint which utilises a POST request really eases workflows by simply querying the data needed in the model, applying the transformations and visualising the results, rather than doing a one-off mathematical transformation from scratch. Not only this feature saves a lot of time in the preparatory and necessary work but it also increases consistency across the various time series and models running off the back of the Macrobond data.  